In [1]:
import tensorflow
import numpy as np

In [2]:
x = np.array([2, -1, 3, -2, 2, 2, 1, -4, 5, 1])
w = np.array([-1, -1, 1, -1, 1, -1, 1, 1, -1, 1])

## Toy Example showing the phenomenal of fool a binary logistic  regression

'p' is the probability of be class 1.
If p > 0.5 it belongs to the class 1 otherwise not.

In [6]:
p = 1/(1+np.power(np.e, -np.dot(x, w)))
print(p)

0.04742587317756679


#### Let's try to foll the classifier

Doing the dot product again we see that suddenly the score becomes 2. This is not surprising: There are 10 dimensions and we’ve tweaked the input by 0.5 in every dimension in such a way that we gain 0.5 in each one, adding up to a total of 5 additional score, rising it from -3 to 2. Now when we look at probability of class 1 we get 1/(1+e^(-2)) = 0.88. That is, we tweaked the original x by a small amount and we improved the class 1 probability from 5% to 88%!

In [11]:
# xad = x + 0.5w gives:
xad = [1.5, -1.5, 3.5, -2.5, 2.5, 1.5, 1.5, -3.5, 4.5, 1.5]
# xad = [1.1, -1.1, 3.1, -2.1, 2.1, 1.1, 1.1, -3.1, 4.1, 1.1]

In [12]:
p = 1/(1+np.power(np.e, -np.dot(xad, w)))
print(p)

0.8807970779778823


## Reproducing the  “Deep MNIST for Experts” tutorial

In [2]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

Epoch 1/5
60000/60000 [==============================] - 11s 186us/step - loss: 0.2195 - acc: 0.9348
Epoch 2/5
60000/60000 [==============================] - 11s 184us/step - loss: 0.0958 - acc: 0.9708
Epoch 3/5
60000/60000 [==============================] - 12s 195us/step - loss: 0.0695 - acc: 0.9779
Epoch 4/5
60000/60000 [==============================] - 12s 197us/step - loss: 0.0537 - acc: 0.9833
Epoch 5/5
10000/10000 [==============================] - 1s 58us/step


[0.06666437025318155, 0.9805]

In [38]:
fisrt_layer = model.layers[1]

#### Selecting samples from the 2

In [39]:
X_samples_2 = x_train[y_train==2]
X_10_samples_2 = X_samples_2[:10]
Y_samples_2 = y_train[y_train==2]

print(X_samples_2.shape)
print(Y_samples_2.shape)
print(X_10_samples_2.shape)

(5958, 28, 28)
(5958,)
(10, 28, 28)
